In [32]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline, AutoTokenizer

In [33]:
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model_name, tokenizer=tokenizer)

Device set to use cpu


In [34]:
def scrape_cnn_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    paragraphs = soup.find_all('p')

    article_text = ''
    for p in paragraphs:
        article_text += p.get_text() + ' '

    return article_text.strip()


In [35]:
def chunk_text_by_tokens(text, max_tokens=900):
    tokens = tokenizer.encode(text)
    
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
        
    return chunks

In [36]:
def summarize(text, max_chunk=800):
    chunks = chunk_text_by_tokens(text)

    summaries = []
    
    words = text.split()
    
    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=120,
            min_length=60,
            do_sample=False
        )
        
        summaries.append(summary[0]['summary_text'])
        
    return ' '.join(summaries)

In [37]:
def summarize_link_cnn(url):
    print('Mengambil artikel')
    article_text = scrape_cnn_article(url)
    summary = summarize(article_text)
    return summary

In [38]:
cnn_url = 'https://edition.cnn.com/2026/01/07/health/dietary-guidelines-rfk-maha'
summary = summarize_link_cnn(cnn_url)
print(summary)

Mengambil artikel
The new recommendations fulfill Kennedy’s promise that they will run only a few pages. Officials say that following the guidance “can help prevent the onset or slow the rate of progression of chronic disease” The latest update includes images of an inverted pyramid that puts meats, cheese and vegetables in the widest part at the top. The 2025-30 dietary guidelines focus on more protein intake than has previously been recommended. The new recommendation is based on body weight: 1.2 to 1.6 grams per kilogram per day. The guidelines also recommend three servings of vegetables and two servings of fruits per day for a typical 2,000-calorie diet. 79% of school meal program directors say they need more funding to reduce reliance on ultraprocessed foods. The American Heart Association calls for Congress to increase funding for school meal programs. The new guidelines may take years to affect school meals. The US Department of Agriculture must first turn the updated guidelines